In [ ]:
import os
import sys

import scipy as sp
from matplotlib import pyplot as plt
import numpy as np

CWD = os.path.abspath("")
sys.path.append(os.path.join(CWD, ".."))
from plt_settings import plt_settings

full_width = 5.5
ratio = 1 / 1.618

In [ ]:
ITERATIONS = int(5e6)
BURN_IN = ITERATIONS // 2

sampling_data = np.load(os.path.join(CWD, "1d_gaussian.npz"))
projlmc_samples = sampling_data["projlmc_x"].flatten()[BURN_IN:]
pdlmc_samples = sampling_data["pdlmc_x"].flatten()[BURN_IN:]
pdlmc_lambda = sampling_data["pdlmc_lambda"]

sampling_data = np.load(os.path.join(CWD, "mirror-langevin/scripts/1d_mirror.npz"))
mirror_samples = sampling_data["mirror_x"][BURN_IN:]

In [ ]:
a, b = 1, 3

# True mean/variance
Z = sp.stats.norm.cdf(b) - sp.stats.norm.cdf(a)
true_mean = sp.integrate.quad(lambda x: x * sp.stats.norm.pdf(x) / Z, a, b)[0]
true_var = sp.integrate.quad(lambda x: x**2 * sp.stats.norm.pdf(x) / Z, a, b)[0] - true_mean**2

# Estimated mean/variance
pdlmc_mean = pdlmc_samples.mean()
pdlmc_var = pdlmc_samples.var()
projlmc_mean = projlmc_samples.mean()
projlmc_var = projlmc_samples.var()
mirror_mean = mirror_samples.mean()
mirror_var = mirror_samples.var()

print(f"Mean/variance of the distribution: {true_mean}/{true_var}")
print(f"Estimated mean/variance via PD-LMC: {pdlmc_mean}/{pdlmc_var}")
print(f"Estimated mean/variance via Proj-LMC: {projlmc_mean}/{projlmc_var}")
print(f"Estimated mean/variance via Mirror LMC: {mirror_mean}/{mirror_var}")


# Out-of-support samples
out_of_support = ((pdlmc_samples < a) | (pdlmc_samples > b)).mean()
print(f"Percentage of out-of-support samples (PD-LMC): {out_of_support * 100:.2f}%")


# Bin egdes overestimation
def count_edge_samples(samples):
    hist, bin_edges = np.histogram(samples[::10], bins=(1, 1.01, 2.99, 3), density=True)
    expected_density_first = (
        (sp.stats.norm.cdf(bin_edges[1]) - sp.stats.norm.cdf(bin_edges[0]))
        / (bin_edges[1] - bin_edges[0])
        / Z
    )
    expected_density_last = (
        (sp.stats.norm.cdf(bin_edges[-2]) - sp.stats.norm.cdf(bin_edges[-1]))
        / (bin_edges[-2] - bin_edges[-1])
        / Z
    )
    density_first = hist[0]
    density_last = hist[-1]
    print(
        f"Overestimation of bin ({bin_edges[0]},{bin_edges[1]}): "
        f"{density_first} / {expected_density_first} ({density_first/expected_density_first * 100:.2f}%)"
    )
    print(
        f"Overestimation of bin ({bin_edges[-2]},{bin_edges[-1]}): "
        f"{density_last} / {expected_density_last} ({density_last/expected_density_last * 100:.2f}%)"
    )


print("PD-LMC:")
count_edge_samples(pdlmc_samples)

print("Proj-LMC:")
count_edge_samples(projlmc_samples)

print(f"Mirror-LMC:")
count_edge_samples(mirror_samples)

In [ ]:
plt_settings["figure.figsize"] = (full_width, ratio / 2.7 * full_width)
xs = np.linspace(a, b, 100)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 3, dpi=300)

    axs[0].hist(pdlmc_samples[::10], bins="auto", density=True)
    axs[0].plot(xs, sp.stats.norm.pdf(xs) / Z, "k--")
    axs[0].set_title("PD-LMC", loc="right", x=0.95, y=0.8, pad=0)

    axs[1].hist(projlmc_samples[::10], bins="auto", density=True)
    axs[1].plot(xs, sp.stats.norm.pdf(xs) / Z, "k--")
    axs[1].set_title("Proj. LMC", loc="right", x=0.95, y=0.8, pad=0)

    axs[2].hist(mirror_samples[::10], bins="auto", density=True)
    axs[2].plot(xs, sp.stats.norm.pdf(xs) / Z, "k--")
    axs[2].set_title("Mirror LMC", loc="right", x=0.95, y=0.8, pad=0)

    for ax in axs:
        ax.set_xlim((0.8, 3.2))
        ax.set_ylim((0, 2.1))
        ax.set_yticks((0, 1, 2))
        ax.grid()
    axs[0].set_ylabel("Density")
    axs[1].set_yticklabels([])
    axs[2].set_yticklabels([])

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    axs.plot(pdlmc_lambda)
    axs.grid()
    axs.set_xlabel("Iteration")
    axs.set_ylabel(r"Dual variable ($\lambda$)")

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)
pdlmc_samples_all = sampling_data["pdlmc_x"].flatten()

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    cum_mean = np.cumsum(
        4 * np.clip((pdlmc_samples_all - a) * (pdlmc_samples_all - b), 0.0, None) - 0.005
    ) / np.arange(1, len(pdlmc_samples_all) + 1)
    axs.plot(cum_mean)
    axs.set_xlim((-1e2, 1e4))
    axs.grid()
    axs.set_xlabel("Iteration")
    axs.set_yticks([0, 5, 10])
    axs.set_ylabel(r"Ergodic constraint slack")
    axs.ticklabel_format(scilimits=(0, 2))

    plt.show()